# Pyne

In [1]:
import numpy as np
import os, sys, yaml, re, glob
from IPython.display import display, Markdown, Latex #Can write latex too!!!!
import pandas as pd

import argparse
import shutil
import math
import matplotlib as mpl

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import scipy.optimize as so
import tables as tb

colors = plt.rcParams['axes.prop_cycle'].by_key()['color'];

#to view entire table (columns and rows)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)  

from pyne import ensdf, nucname, nuc_data, data

/home/anton/anaconda3/envs/pyne_debug/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.rxname is not yet QA compliant.
  return f(*args, **kwds)
/home/anton/anaconda3/envs/pyne_debug/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/home/anton/anaconda3/envs/pyne_debug/lib/python3.6/site-packages/pyne/ensdf.py:17: QAWarning: pyne.ensdf is not yet QA compliant.
  warn(__name__ + " is not yet QA compliant.", QAWarning)


Declaring classes and parameters

In [2]:
def GetQ(E, m):
    return np.multiply(E, (1 + np.divide(4.,m-4)));

class ADecay:
    def __init__(self, data):
        if isinstance(data[0], int):
            self.mother_id = int(data[0])
            #print('Setting mother_id = ', self.mother_id)
            if self.mother_id < 0:
                self.mother_id = -self.mother_id
                raise ValueError('Parent id = negative')
        else:
            self.mother_id = None
        if isinstance(data[1], int):
            self.child_id = int(data[1])            
            #print('Setting child_id = ', self.child_id)
            if self.child_id < 0:
                self.child_id = -self.child_id
                raise ValueError('Child id = negative')
        else:
            self.child_id = None
        if data[3] is not None:
            self.half_life = float(data[3])
        else:
            self.half_life = np.nan
        if data[4] is not None:
            self.half_life_error = float(data[4])
        else:
            self.half_life_error = np.nan
        if data[5] is not None:
            self.branch_ratio = float(data[5])
        else:
            self.branch_ratio = np.nan
        if data[6] is not None:
            self.branch_ratio_error = float(data[6])
        else:
            self.branch_ratio_error = np.nan

        if self.mother_id is not None:
            self.A = int(nucname.anum(self.mother_id))
            self.Z = int(nucname.znum(self.mother_id))
            self.name = nucname.name(self.mother_id)
            self.Zname = nucname.zz_name[self.Z]
        else:
            #print('data[0] = ', data[0])
            rec = ensdf_mod.extra_SHE.match(data[0])
            self.A = int(rec.group(1))
            self.Z = int(ensdf_mod.d_extra_SHE[rec.group(2)])
            self.name = rec.group(2)+rec.group(1)
            self.Zname = rec.group(2)
        if self.child_id is not None:
            self.a = int(nucname.anum(self.child_id))
            self.z = int(nucname.znum(self.child_id))
            self.child_name = nucname.name(self.child_id)
            self.child_Zname = nucname.zz_name[self.z]
        else:
            #print('data[1] = ', data[1])
            self.a = int(data[1][1])
            self.z = int(ensdf_mod.d_extra_SHE[data[1][0]])
            self.child_name = nucname.name(self.child_id)
            self.child_Zname = nucname.zz_name[self.z]
        
        self.decay_mode = ''
        if self.A - self.a == 4:
            self.decay_mode = 'Alpha'
        elif self.A == self.a and self.Z == self.z+1:
            self.decay_mode = 'Beta_minus'
        elif self.A == self.a and self.Z == self.z-1:
            self.decay_mode = 'Beta_plus_or_EC'
        elif self.A == self.a and self.Z == self.z:
            self.decay_mode = 'gamma'
        #else:
            #print("WARNING NO DECAY MODE FOR", self)
        
        if self.name[-1] == 'M':
            self.isomer = True
        else: 
            self.isomer = False
            
        self.nbranches = 0
        self.E_alphas = None
        self.I_alphas = None
        self.child_levels = None
        self.child_l = None
        self.child_l_tent = None
        self.l_alphas = None
        self.l_alphas_tent = None
        self.child_parity = None
        self.child_parity_tent = None
        print(data[12])
        if data[12] is not None:
            self.nbranches = len(data[12])
            self.E_alphas = np.full(self.nbranches, np.nan)
            self.Q_alphas = np.full(self.nbranches, np.nan)
            self.I_alphas = np.full(self.nbranches, np.nan)
            #self.child_levels = np.full(self.nbranches, np.nan)
            #self.child_l = np.full(self.nbranches, np.nan)
            #self.l_alphas = np.full(self.nbranches, np.nan)
            #self.l_alphas_tent = np.full(self.nbranches, np.nan)
            #self.child_l_tent = np.full(self.nbranches, np.nan)
            #self.child_parity = np.full(self.nbranches, np.nan)
            #self.child_parity_tent = np.full(self.nbranches, np.nan)
            for i, v in enumerate(data[12]):
                self.E_alphas[i] = v[2]
                self.Q_alphas[i] = GetQ(self.E_alphas[i]*.001, self.A)
                if v[3] is not None:
                    self.I_alphas[i] = v[3]*1e-2
                #if v[4] is not None:
                    #self.child_levels[i] = v[4]*1e-3
                #if v[5] is not None:
                    #self.child_l[i], self.child_l_tent[i] = v[5]
                #if v[6] is not None:
                    #self.child_parity[i], self.child_parity_tent[i] = v[6]
                #self.l_alphas[i] = np.abs(self.l_parent-self.child_l[i])
                #if any(np.isnan([self.child_l[i], self.l_parent])):
                    #self.l_alphas_tent[i] = np.nan
                #else:
                    #self.l_alphas_tent[i] = bool(self.l_parent_tent)|bool(self.child_l_tent[i])
            
     #overload string representation which is e.g. invoked in print()
    def __str__(self):
        string = self.name + ':\n'
        string += 'Decay: ' + self.name + ' -> ' + self.child_name + ', ' + self.decay_mode + '\n'
        string += 'half-life = ' + str(self.half_life) + '+-' + str(self.half_life_error) + '\n'
        string += 'branch_ratio = ' + str(self.branch_ratio) + '+-' + str(self.branch_ratio_error) + '\n'
        string += "E_alphas = " + np.str(self.E_alphas) + '\n'
        string += "Q_alphas = " + np.str(self.Q_alphas) + '\n'
        string += "I_alphas = " + np.str(self.I_alphas) + '\n'
        return string
        
    #overload representation which is important if the object is retrieved from list
    def __repr__(self):
        return str(self)

Read nuclear data and import into variables

In [3]:
import ensdf_mod
build_dir = "../pyne_source/scripts/build_nuc_data"
build_dir = os.path.join(build_dir, 'ENSDF')

decay_data = []
files = sorted([f for f in glob.glob(os.path.join(build_dir, 'ensdf.*'))])
for f in files:
    print("    parsing decay data from {0}".format(f))
    decay_data = ensdf_mod.decays(f, decay_data)


/home/anton/Documents/PhD/pyne/pyne_debug/ensdf_mod.py:17: QAWarning: ensdf_mod is not yet QA compliant.
  warn(__name__ + " is not yet QA compliant.", QAWarning)
/home/anton/Documents/PhD/pyne/pyne_debug/ensdf_mod.py:67: UserWarning: Neutron data not supported!
  warn('Neutron data not supported!')


    parsing decay data from ../pyne_source/scripts/build_nuc_data/ENSDF/ensdf.001
('  1H ', '1NN B- DECAY (613.9 S)        ', '                          ', '05NDS  ', '  2006')
    parsing decay data from ../pyne_source/scripts/build_nuc_data/ENSDF/ensdf.002
    parsing decay data from ../pyne_source/scripts/build_nuc_data/ENSDF/ensdf.003
('  3HE', '3H B- DECAY                   ', '                          ', '15NDS  ', '  2015')
    parsing decay data from ../pyne_source/scripts/build_nuc_data/ENSDF/ensdf.004
    parsing decay data from ../pyne_source/scripts/build_nuc_data/ENSDF/ensdf.005
('  5LI', '9C B+A DECAY                  ', '1988MI03,2000GE09,2001BE51', '       ', '      ')
    parsing decay data from ../pyne_source/scripts/build_nuc_data/ENSDF/ensdf.006
('  6LI', '6HE B- DECAY                  ', '2002TI10                  ', '02NP   ', '  2002')
    parsing decay data from ../pyne_source/scripts/build_nuc_data/ENSDF/ensdf.007
('  7LI', '7BE EC DECAY                  ', '2

(' 29NA', '29NE B- DECAY                 ', '2005TR05,2006TR02,1999RE16', '12NDS  ', '  2012')
(' 29MG', '29NA B- DECAY                 ', '1987BA46                  ', '12NDS  ', '  2012')
(' 29MG', '30NA B-N DECAY                ', '1989BA07                  ', '12NDS  ', '  2012')
(' 29MG', '31NA B-2N DECAY               ', '1993KL02                  ', '12NDS  ', '  2012')
(' 29AL', '29MG B- DECAY                 ', '1984GU19,1973GO34,1979DE02', '12NDS  ', '  2012')
(' 29SI', '29AL B- DECAY                 ', '1982AL27,1970JO06,1969HA51', '12NDS  ', '  2012')
(' 29SI', '29P B+ DECAY                  ', '1980WI13                  ', '12NDS  ', '  2012')
(' 29P ', '29S B+ DECAY                  ', '1979VI01,1985ZH05         ', '12NDS  ', '  2012')
(' 29P ', '31AR B+2P DECAY               ', '1998AX02                  ', '12NDS  ', '  2012')
(' 29CL', '30AR P DECAY:(2+) STATE       ', '2015MU13                  ', 'ENSDF  ', '  2016')
    parsing decay data from ../pyne_source/scripts

(' 39S ', '39P B- DECAY (0.28 S)         ', '1998WIZV,1988MU08         ', '06NDS  ', '  2006')
(' 39S ', '40P B-N DECAY (153 MS)        ', '2001WI21,1989LE16         ', '06NDS  ', '  2006')
(' 39CL', '39S B- DECAY (11.5 S)         ', '1980HI01                  ', '06NDS  ', '  2006')
(' 39AR', '39CL B- DECAY (55.6 M)        ', '1987WA12                  ', '06NDS  ', '  2006')
(' 39K ', '39AR B- DECAY (269 Y)         ', '1965ST09,1950BR66         ', '06NDS  ', '  2006')
(' 39K ', '39CA EC DECAY (859.6 MS)      ', '1994HA07,1984AD01         ', '06NDS  ', '  2006')
(' 39K ', '40SC ECP DECAY (182.3 MS)     ', '1982HO09                  ', '06NDS  ', '  2006')
(' 39CA', '40TI ECP DECAY (53.3 MS)      ', '1998BH12,1998LI46,1990DE43', '06NDS  ', '  2006')
(' 39SC', '39TI EC DECAY (31 MS)         ', '1992MO15,2001GI01         ', '06NDS  ', '  2006')
    parsing decay data from ../pyne_source/scripts/build_nuc_data/ENSDF/ensdf.040
(' 40S ', '40P B- DECAY (150 MS)         ', '2001WI21          

(' 49K ', '50AR B-N DECAY (85 MS)        ', '2003WE09                  ', 'ENSDF  ', '  2011')
(' 49CA', '49K B- DECAY                  ', '                          ', '08NDS  ', '  2008')
(' 49CA', '50K B-N DECAY                 ', '1983RAZR,1982CA04,1998BA80', '08NDS  ', '  2008')
(' 49SC', '49CA B- DECAY                 ', '1971EI03,2003BE37         ', '08NDS  ', '  2008')
(' 49TI', '49SC B- DECAY                 ', '1971EI03                  ', '08NDS  ', '  2008')
(' 49TI', '49V EC DECAY                  ', '1983HU03,1983BA08,1972KR12', '08NDS  ', '  2008')
(' 49V ', '49CR B+ DECAY                 ', '1975JA17,1974TA11         ', '08NDS  ', '  2008')
(' 49CR', '49MN B+ DECAY                 ', '1989HO13,1980HA12         ', '08NDS  ', '  2008')
(' 49MN', '49FE B+ DECAY: PARTIAL        ', '1996FA09,2007DO17         ', '08NDS  ', '  2008')
(' 49MN', '50CO B+P DECAY: PARTIAL       ', '1996FA09,2007DO17         ', '08NDS  ', '  2008')
(' 49FE', '50NI B+P DECAY: PARTIAL       ', '2003M

...

('206PO', '210RN A DECAY                 ', '                          ', '08NDS  ', '  2008')
('206AT', '206RN EC DECAY                ', '1975ZEZY                  ', '08NDS  ', '  2008')
('206AT', '210FR A DECAY                 ', '2005KU06                  ', '08NDS  ', '  2008')
('206RN', '206FR EC DECAY                ', '1981RI02                  ', '08NDS  ', '  2008')
('206RN', '210RA A DECAY                 ', '                          ', '08NDS  ', '  2008')
('206FR', '210AC A DECAY                 ', '1968VA04,2000He17         ', '08NDS  ', '  2008')
('206RA', '210TH A DECAY                 ', '1995Uu01                  ', '08NDS  ', '  2008')
    parsing decay data from ../pyne_source/scripts/build_nuc_data/ENSDF/ensdf.207
('207TL', '207HG B- DECAY                ', '1981JOZW                  ', '11NDS  ', '  2011')
('207TL', '207TL IT DECAY (1.33 S)       ', '1965EC02                  ', '11NDS  ', '  2011')
('207TL', '211BI A DECAY                 ', '1991RY01,1976BL13 

!%%%%%%%%%%%%%
('220AC', '          ', '  ', '  0.5   ', 'LT', '  660   ', 'GT') 
 220AC  A               0.5   LT  660   GT                                        


%%%%%%%%%%%%%
E_ALPHA == None 
!%%%%%%%%%%%%%
('220AC', '          ', '  ', '  0.7   ', 'LT', '  370   ', 'GT') 
 220AC  A               0.7   LT  370   GT                                        

('220TH', '224U A DECAY                  ', '2014LO10                  ', 'ENSDF  ', '  2014')
    parsing decay data from ../pyne_source/scripts/build_nuc_data/ENSDF/ensdf.221
('221RN', '225RA A DECAY                 ', '2000LI37                  ', '07NDS  ', '  2007')
('221FR', '221RN B- DECAY                ', '1977Vy02                  ', '07NDS  ', '  2007')
('221FR', '225AC A DECAY                 ', '2000AR23,2003KU44         ', '07NDS  ', '  2007')
('221RA', '225TH A DECAY                 ', '                          ', '07NDS  ', '  2007')
('221AC', '225PA A DECAY                 ', '                          ', '07ND

('229RA', '229FR B- DECAY                ', '1999FR33                  ', '08NDS  ', '  2008')
('229AC', '229RA B- DECAY                ', '1997Ya05,1975We23         ', '08NDS  ', '  2008')
('229TH', '229AC B- DECAY                ', '2002GU15,2006RU07         ', '08NDS  ', '  2008')
('229TH', '229TH IT DECAY: 7.6 EV        ', '1994HE08,2007BE16         ', '08NDS  ', '  2008')
('229TH', '229PA EC DECAY                ', '1987AH05                  ', '08NDS  ', '  2008')
('229TH', '233U A DECAY                  ', '2003BA78,2007BE16         ', '08NDS  ', '  2008')

%%%%%%%%%%%%%
E_ALPHA == None 
!%%%%%%%%%%%%%
('229TH', '          ', '  ', '        ', '  ', '        ', '  ') 
 229TH  A                                                                         


%%%%%%%%%%%%%
E_ALPHA == None 
!%%%%%%%%%%%%%
('229TH', '          ', '  ', '        ', '  ', '        ', '  ') 
 229TH  A                                                                         


%%%%%%%%%%%%%
E_ALPHA == None 
!%

('236U ', '236NP EC DECAY (153E+3 Y)     ', '1983AH02,1981LI30         ', '06NDS  ', '  2006')
('236U ', '236NP EC DECAY (22.5 H)       ', '1969LE05                  ', '06NDS  ', '  2006')
('236U ', '240PU A DECAY                 ', '1981HE16,1986LOZT         ', '06NDS  ', '  2006')
('236NP', '240AM A DECAY                 ', '1970GO42                  ', '06NDS  ', '  2006')
('236PU', '236NP B- DECAY (153E+3 Y)     ', '1981LI30,1983AH02         ', '06NDS  ', '  2006')
('236PU', '236NP B- DECAY (22.5 H)       ', '1956GR11,1959GI58         ', '06NDS  ', '  2006')
('236PU', '236AM EC DECAY (3.6 M)        ', '2005AS01                  ', '06NDS  ', '  2006')
('236PU', '236AM EC DECAY (2.9 M)        ', '2005AS01                  ', '06NDS  ', '  2006')
('236PU', '240CM A DECAY                 ', '                          ', '06NDS  ', '  2006')
('236CM', '240CF A DECAY                 ', '                          ', '06NDS  ', '  2006')
('236BK', '240ES A DECAY                 ', '2017K

('245AM', '245PU B- DECAY                ', '1968Da02                  ', '11NDS  ', '  2011')
('245AM', '249BK A DECAY                 ', '                          ', '11NDS  ', '  2011')
('245CM', '245AM B- DECAY                ', '1967BU09                  ', '11NDS  ', '  2011')
('245CM', '245BK EC DECAY                ', '1976AH03                  ', '11NDS  ', '  2011')
('245CM', '249CF A DECAY                 ', '1982AH04,1997AR31         ', '11NDS  ', '  2011')
('245BK', '245CF EC DECAY                ', '                          ', '11NDS  ', '  2011')
('245BK', '249ES A DECAY                 ', '                          ', '11NDS  ', '  2011')
('245CF', '245ES EC DECAY                ', '                          ', '11NDS  ', '  2011')
('245CF', '249FM A DECAY                 ', '                          ', '11NDS  ', '  2011')
('245ES', '249MD A DECAY                 ', '2005HE27,2008GA25         ', '11NDS  ', '  2011')
    parsing decay data from ../pyne_source/scripts

In [4]:
structured_data = []
index = np.asarray([])
for i, v in enumerate(decay_data):
    try:
        a = ADecay(v)
        if a.name == '':
            print('\ndata = ', v, '\n')
            print(a, "\n")
            break
        if a.decay_mode == 'Alpha':
            for j in range(a.nbranches):
                structured_data.append([int(a.A), int(a.Z), int(a.isomer), float(a.E_alphas[j]), float(a.half_life), float(a.branch_ratio), float(a.I_alphas[j])])
                index = np.append(index, np.asarray([a.name]))
    except RuntimeError:
        print('RuntimeError: Nucleus not found!')
    except ValueError:
        print('Value error')

print('Shape index, data = ', np.shape(index.reshape(len(index), 1)), np.shape(np.asarray(structured_data)))

RuntimeError: Nucleus not found!
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[[731620000, 711580000, 4883.0, 100.0]]
[[741620000, 721580000, 5536.0, 100.0]]
[]
[[751620000, 731580000, 6086.0, 100.0]]
[[751620001, 731580001, 6119.0, 100.0], [751620001, 731580002, 6037.0, None]]
[[761620000, 741580000, 6602.0, 100.0]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[731630000, 711590000, 4633.0, None]]
[[741630000, 721590000, 5384.0, None]]
[[751630000, 731590000, 5870.0, None]]
[[751630001, 731590001, 5919.0, None]]
[[761630000, 741590000, 6513.0, None]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[741640000, 721600000, 5149.6, None]]
[[751640000, 731600000, 5782.0, 100.0]]
[[761640000, 741600000, 6321.0, None]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[741650000, 721610000, 4909.0, 100.0]]
[[751650001, 731610000, 5518.0, 100.0]]
[[761650000, 741610000, 6183.0, 100.0]]
[[771650001, 751610001, 6715.0, 100.0]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[741660000, 721620000, 4739.0, 100.0]]
[[751660000, 731620000, 5522.0, 100.0]]
[[761660000, 741620000, 5993.0, 100.0]]
[

[[831860001, 811820000, 7369.0, 2.0], [831860001, 811820000, 7263.0, 100.0]]
[[841860000, 821820000, 8320.0, None]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[791870000, 771830000, 4690.0, 100.0]]
[]
[]
[[801870000, 781830002, 5035.0, 100.0]]
[[801870000, 781830010, 4870.0, 100.0]]
[]
[[811870002, 791830000, 5536.0, 9.0], [811870002, 791830001, 5524.0, 91.0]]
[[821870000, 801830005, 6077.0, 100.0]]
[[821870001, 801830002, 6194.0, 59.7], [821870001, 801830008, 5993.0, 40.3]]
[[831870000, 811830000, 7612.0, 9.0], [831870000, 811830001, 7342.0, 3.0], [831870000, 811830002, 7000.0, 88.0]]
[[831870002, 811830000, 7721.0, 100.0]]
[[841870000, 821830000, 7808.0, 2.0], [841870000, 821830002, 7528.0, 100.0]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[781880000, 761840000, 3919.0, 95.0]]
[]
[]
[]
[[801880000, 781840000, 4610.0, 96.0]]
[]
[]
[]
[[821880000, 801840000, 5983.0, 99.0], [821880000, 801840001, 5633.0, 0.095], [821880000, 801840002, 5614.0, 1.0]]
[[831880000, 811840000, 7302.0, 3.3], [831880000, 81184000

[]
[]
[[852070000, 832030000, 5758.0, 100.0]]
[]
[]
[[862070000, 842030000, 6131.0, 99.24], [862070000, 842030001, 6072.0, 0.66], [862070000, 842030002, 6000.0, 0.1]]
[[872070000, 852030000, 6767.0, 100.0]]
[[882070000, 862030000, 7131.0, 100.0]]
[[882070001, 862030001, 7323.0, 100.0]]
[[892070000, 872030000, 7693.0, 100.0]]
[]
[]
[]
[]
[]
[]
[]
[[842080000, 822040000, 5114.9, 99.99976], [842080000, 822040001, 4220.0, 0.00024]]
[]
[]
[[852080000, 832040000, 5640.3, 96.9], [852080000, 832040001, None, None], [852080000, 832040002, 5626.0, 2.14], [852080000, 832040003, 5586.0, 0.87]]
[]
[[862080000, 842040000, 6140.1, 99.953], [862080000, 842040001, 5469.1, 0.047]]
[]
[[872080000, 852040000, 6641.0, 100.0]]
[[882080000, 862040000, 7133.0, 100.0]]
[[892080000, 872040000, 7572.0, 100.0]]
[[892080001, 872040002, 7758.0, 100.0]]
[]
[]
[]
[]
[[832090000, 812050000, 3077.0, 99.92], [832090000, 812050001, 2876.0, 0.08]]
[]
[]
[[842090000, 822050000, 4885.0, 19.8], [842090000, 822050001, 4883.0,

[[892210000, 872170000, 7645.0, 68.0], [892210000, 872170001, 7440.0, 21.0], [892210000, 872170002, 7377.0, 9.0], [892210000, 872170004, 7170.0, 2.4]]
[[902210000, 882170000, 8470.0, 39.0], [902210000, 882170001, 8145.0, 56.0], [902210000, 882170004, 7730.0, 5.0]]
[[912210000, 892170000, 9075.0, 100.0]]
[]
[[862220000, 842180000, 5489.48, 99.92], [862220000, 842180001, 4986.0, 0.078], [862220000, 842180002, 4826.0, 0.0005]]
[[882220000, 862180000, 6559.0, 96.9], [882220000, 862180001, 6241.0, 3.05], [882220000, 862180002, 5918.0, 0.0041], [882220000, 862180003, 5777.0, 0.0041], [882220000, 862180004, 5734.0, 0.0042]]
[[892220000, 872180000, 7008.6, 94.0], [892220000, 872180001, 6963.0, 6.0]]
[[892220000, 872180000, 7000.0, 15.0], [892220000, 872180000, 6970.0, 8.0], [892220000, 872180000, 6890.0, 15.0], [892220000, 872180000, 6840.0, 10.0], [892220000, 872180000, 6810.0, 27.0], [892220000, 872180000, 6750.0, 15.0], [892220000, 872180000, 6710.0, 8.0], [892220000, 872180000, 6460.0, 2.0

[[922270000, 902230001, 7060.0, 20.0], [922270000, 902230004, 6905.0, 14.0], [922270000, 902230007, 6860.0, 50.0], [922270000, 902230008, 6740.0, 16.0]]
[[932270000, 912230000, 7677.0, None], [932270000, 912230001, 7650.0, None]]
[]
[]
[]
[[902280000, 882240000, 5423.15, 73.4], [902280000, 882240001, 5340.36, 26.0], [902280000, 882240002, 5211.0, 0.408], [902280000, 882240003, 5173.0, 0.218], [902280000, 882240004, 5138.0, 0.036], [902280000, 882240005, None, 1e-05], [902280000, 882240006, None, 2.4e-05], [902280000, 882240010, None, 1.7e-05], [902280000, 882240012, None, 4.6e-06]]
[[912280000, 892240003, 6126.0, 1.0], [912280000, 892240005, 6117.0, 9.9], [912280000, 892240010, 6104.0, 11.3], [912280000, 892240011, 6089.0, 2.2], [912280000, 892240014, 6076.0, None], [912280000, 892240015, 6076.0, 19.5], [912280000, 892240017, 6065.0, 1.0], [912280000, 892240020, 6052.0, 0.7], [912280000, 892240022, 6047.0, None], [912280000, 892240023, 6047.0, 0.3], [912280000, 892240024, 6040.0, 2.2],

[[1012500000, 992460000, 7750.0, 75.0], [1012500000, 992460000, 7833.0, 25.0]]
[]
[[982510000, 962470000, 6078.0, 2.6], [982510000, 962470001, 6017.0, 12.5], [982510000, 962470002, 5946.0, 0.6], [982510000, 962470003, None, None], [982510000, 962470004, 5854.0, 27.6], [982510000, 962470005, 5817.0, 4.0], [982510000, 962470006, 5798.0, 2.5], [982510000, 962470009, 5766.0, 3.6], [982510000, 962470011, 5738.0, 0.8], [982510000, 962470015, 5679.3, 35.4], [982510000, 962470018, 5651.0, 3.3], [982510000, 962470020, 5635.0, 4.9], [982510000, 962470022, 5569.0, 1.9], [982510000, 962470023, 5567.0, 1.9], [982510000, 962470026, 5505.0, 0.27]]
[]
[[992510000, 972470000, 6492.0, 81.0], [992510000, 972470001, 6462.0, 9.4], [992510000, 972470002, 6452.0, 3.3], [992510000, 972470003, 6422.0, 3.0], [992510000, 972470004, 6410.0, 3.3]]
[[1002510000, 982470000, 7306.0, 1.5], [1002510000, 982470001, 7252.0, 0.93], [1002510000, 982470002, 7185.0, 0.29], [1002510000, 982470003, 7107.0, 0.05], [1002510000, 

[]
[[1012550000, 992510002, 7752.0, 1.0], [1012550000, 992510003, 7714.0, 1.0], [1012550000, 992510010, 7327.0, 93.0], [1012550000, 992510012, 7274.0, 5.0]]
[[1022550000, 1002510000, 8296.0, 1.2], [1022550000, 1002510001, 8233.0, 6.9], [1022550000, 1002510002, 8100.0, 29.7], [1022550000, 1002510003, 8057.0, 10.3], [1022550000, 1002510004, 8001.0, 6.8], [1022550000, 1002510005, None, 0.5], [1022550000, 1002510007, 7909.0, 16.6], [1022550000, 1002510008, 7842.0, 4.3], [1022550000, 1002510009, 7748.0, 18.4], [1022550000, 1002510010, 7726.0, 2.7], [1022550000, 1002510011, 7702.0, 2.7]]
[[1032550000, 1012510000, 8420.0, 5.0], [1032550000, 1012510001, 8365.0, 93.3], [1032550000, 1012510003, 8290.0, 1.7]]
[[1032550001, 1012510000, 8457.0, 93.0], [1032550001, 1012510003, 8322.0, 7.0]]
[[1042550000, 1022510000, 8575.0, 1.0], [1042550000, 1022510000, 8646.0, 1.5], [1042550000, 1022510000, 8678.0, 3.0], [1042550000, 1022510000, 8906.0, 2.5], [1042550000, 1022510003, 8716.0, 92.0]]
[]
[[1002560000

In [5]:
ind_names = ["Name"]

column_names = ["A", "Z", "Isomer?", "E_alpha (keV)", "t1/2 (s)", "I_alpha (%)", "I_alpha_branch (%)"]

In [6]:
index = index.reshape(len(index), 1)
structured_data = np.asarray(structured_data)
#print(alpha_data)
index = pd.MultiIndex.from_arrays(index.T, names=ind_names)
structured_data_table = dict(zip(column_names, structured_data.T))
df = pd.DataFrame(data=structured_data_table, index=index, columns=column_names)
#to view entire table (columns and rows)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)
for i in range(3, len(df.columns)):
    df.iloc[:,i] = pd.to_numeric(df.iloc[:,i])
display(df)

,A,Z,Isomer?,E_alpha (keV),t1/2 (s),I_alpha (%),I_alpha_branch (%)
Name,,,,,,,
Ni58M,58.0,28.0,1.0,7447.00,NaN,1.000000e+00,2.600000e-02
Te105,105.0,52.0,0.0,4703.00,6.200000e-07,1.000000e+00,1.000000e+00
Te106,106.0,52.0,0.0,4128.00,6.000000e-05,1.000000e+00,1.000000e+00
Te107,107.0,52.0,0.0,3853.00,3.100000e-03,7.000000e-01,9.953000e-01
Te107,107.0,52.0,0.0,3690.00,3.100000e-03,7.000000e-01,4.700000e-03
Te108,108.0,52.0,0.0,3318.00,2.100000e+00,4.900000e-01,1.000000e+00
Te109,109.0,52.0,0.0,3080.00,4.600000e+00,4.000000e-02,1.000000e+00
Xe109,109.0,54.0,0.0,4062.00,1.300000e-02,1.000000e+00,7.000000e-01
Xe109,109.0,54.0,0.0,3918.00,1.300000e-02,1.000000e+00,3.000000e-01


In [7]:
display(df[df['E_alpha (keV)'].isnull()])

,A,Z,Isomer?,E_alpha (keV),t1/2 (s),I_alpha (%),I_alpha_branch (%)
Name,,,,,,,
Nd144,144.0,60.0,0.0,NaN,7.226690e+22,1.000000e+00,1.000000e+00
Er156,156.0,68.0,0.0,NaN,1.170000e+03,1.200000e-07,1.000000e+00
Ta161,161.0,73.0,0.0,NaN,3.080000e+00,7.000000e-02,NaN
At208,208.0,85.0,0.0,NaN,5.868000e+03,5.500000e-03,NaN
Bi210M,210.0,83.0,1.0,NaN,9.593510e+13,1.000000e+00,1.000000e-06
Fr212,212.0,87.0,0.0,NaN,1.200000e+03,4.300000e-01,1.000000e-02
Fr212,212.0,87.0,0.0,NaN,1.200000e+03,4.300000e-01,NaN
Fr220,220.0,87.0,0.0,NaN,2.740000e+01,9.965000e-01,6.000000e-03
Ac224,224.0,89.0,0.0,NaN,1.000800e+04,9.100000e-02,4.000000e-03


In [8]:
display(df.reset_index())
df = df.reset_index()

,Name,A,Z,Isomer?,E_alpha (keV),t1/2 (s),I_alpha (%),I_alpha_branch (%)
0,Ni58M,58.0,28.0,1.0,7447.00,NaN,1.000000e+00,2.600000e-02
1,Te105,105.0,52.0,0.0,4703.00,6.200000e-07,1.000000e+00,1.000000e+00
2,Te106,106.0,52.0,0.0,4128.00,6.000000e-05,1.000000e+00,1.000000e+00
3,Te107,107.0,52.0,0.0,3853.00,3.100000e-03,7.000000e-01,9.953000e-01
4,Te107,107.0,52.0,0.0,3690.00,3.100000e-03,7.000000e-01,4.700000e-03
5,Te108,108.0,52.0,0.0,3318.00,2.100000e+00,4.900000e-01,1.000000e+00
6,Te109,109.0,52.0,0.0,3080.00,4.600000e+00,4.000000e-02,1.000000e+00
7,Xe109,109.0,54.0,0.0,4062.00,1.300000e-02,1.000000e+00,7.000000e-01
8,Xe109,109.0,54.0,0.0,3918.00,1.300000e-02,1.000000e+00,3.000000e-01
9,Te110,110.0,52.0,0.0,2624.00,1.860000e+01,6.700000e-06,1.000000e+00


## Data corrections

For at least three of SHN the $Q_{\alpha}$ is a factor of 1000 too low. Here it is corrected:

In [9]:
display(df[df['Q_alpha (MeV)'] < 1]['Q_alpha (MeV)'])
df.loc[df['Q_alpha (MeV)'] < 1, 'Q_alpha (MeV)'] = df[df['Q_alpha (MeV)'] < 1]['Q_alpha (MeV)']*1e3

KeyError: 'Q_alpha (MeV)'

In [ ]:
#removing 17=ecAlpha decay, 38, 1128 = almost exact duplicate & 547 = Pb202 with unknown branching ratio
df = df.drop([17, 38, 547, 1128])

In [ ]:
display('Duplicates: ', df[df.duplicated(keep=False)]) #displaying which are removed
df = df.drop_duplicates(keep='first')

In [ ]:
display(df)

## Writing data to binary file

In [ ]:
df.to_pickle('AlphaDecaysDataFrame3.p')